In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Table Tennis Rubber Testing Utility

Make sure to go to "Cell --> Run All" after opening this file

In [2]:
%matplotlib inline

In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from IPython.display import HTML

### First, tell me about the shot you're trying to emulate

In [14]:
style = {'description_width': 'initial'}

v_ball = widgets.BoundedFloatText(
    value = 10.0,
    min = 0.0,
    description = "Velocity of ball (m/s):",
    continuous_update = False,
    style = style)

angle_ball = widgets.BoundedFloatText(
    value = 30.0,
    min = -89.0,
    max = 89.0,
    description = "Angle of ball's path (deg):",
    continuous_update = False,
    style = style)

v_paddle = widgets.BoundedFloatText(
    value = 10.0,
    min = 0.0,
    description = "Velocity of paddle (m/s):",
    continuous_update = False,
    style = style)

angle_paddle = widgets.BoundedFloatText(
    value = 20.0,
    min = -90.0,
    max = 90.0,
    description = "Angle of paddle's path (deg):",
    continuous_update = False,
    style = style)

closed_paddle = widgets.BoundedFloatText(
    value = 20.0,
    min = -89.9,
    max = 89.9,
    description = "How closed is the paddle (deg):",
    continuous_update = False,
    style = style)

ui = widgets.VBox([v_ball, angle_ball, v_paddle, angle_paddle, closed_paddle])

def deg2rad(deg):
    return np.pi / 180.0 * deg

def shot_setup(v_ball, angle_ball, v_paddle, angle_paddle, closed_paddle):
    
    # First, we do some plotting to sanity check our inputs...
    fig, ax = plt.subplots(figsize = (8, 4))
    
    ax.axis('equal')
    
    x_table = np.array([-1, -0.5])
    y_table = 0.0 * x_table
    # ax.plot(x_table, y_table, color = 'black', linewidth = 2.0)
    
    x_ball_prev = np.linspace(-1, -0.6, 101)
    y_ball_prev = -(x_ball_prev + 0.6) * (x_ball_prev + 1.5)
    #ax.plot(x_ball_prev, y_ball_prev, color = 'C0')
    
    t = np.linspace(-0.25, 0, 101)
    x_p = -t[0] * v_ball * np.cos(deg2rad(angle_ball)) - 0.6
    y_p = 0.5 * 9.8 * t[0]**2.0 - v_ball * np.sin(deg2rad(angle_ball)) * t[0]
    x_ball_after = v_ball * np.cos(deg2rad(angle_ball)) * t + x_p
    y_ball_after = v_ball * np.sin(deg2rad(angle_ball)) * t - 0.5 * 9.8 * t**2.0 + y_p 
    ax.plot(x_ball_after, y_ball_after, color = 'C0')
    
    x_paddle = np.array([x_p, x_p + 1])
    y_star = y_p - np.tan(deg2rad(angle_paddle))
    y_paddle = (y_star - y_p) * (x_paddle - x_p) + y_p
    ax.plot(x_paddle, y_paddle, color = 'C1')
    
    x_bat = np.array([x_p - 0.15 / 2 * np.sin(deg2rad(closed_paddle)),
                      x_p + 0.15 / 2 * np.sin(deg2rad(closed_paddle))])
    y_bat = np.array([y_p + 0.15 / 2 * np.cos(deg2rad(closed_paddle)),
                      y_p - 0.15 / 2 * np.cos(deg2rad(closed_paddle))])
    ax.plot(x_bat, y_bat, color = 'C1', linewidth = 4.0)
    
    ball = Circle((x_p, y_p), radius = 0.02, color = 'C0', zorder = 2)
    ax.add_artist(ball)
    
#    ax.set_xlim([-1, 1])
#    ax.set_ylim([-0.2, 0.6])
    ax.set_xticks([])
    ax.set_yticks([])

    for spine in ax.spines.values():
        spine.set_visible(False)
    
    plt.show()


    # Now we do MATH!

    # in the world's reference frame
    ball_vec = np.matrix([[v_ball * np.cos(deg2rad(angle_ball))],
                          [v_ball * np.sin(deg2rad(angle_ball))]])
    paddle_vec = np.matrix([[-v_paddle * np.cos(deg2rad(angle_paddle))],
                            [v_paddle * np.sin(deg2rad(angle_paddle))]])
    
    # now we move the vectors to the paddle's reference frame
    paddle_in_p_vec = paddle_vec - paddle_vec
    ball_in_p_vec = ball_vec - paddle_vec
    
    ball_traj_angle = 90.0 - closed_paddle - 180.0 / np.pi * np.arctan(-ball_in_p_vec[1,0]/ball_in_p_vec[0,0])
    ball_traj_speed = (ball_in_p_vec.T * ball_in_p_vec)[0,0]**0.5

    
    print("This is the same as the following situation:")
    print("The paddle is not moving")
    print("The ball has a speed of " + '{:.2f}'.format(ball_traj_speed) + " m/s")
    print("The angle between the paddle and the ball is " + '{:.1f}'.format(ball_traj_angle) + " deg")
    
    # Now display the confirmation plot
    fig, ax = plt.subplots(figsize = (3, 4))
    
    ax.axis('equal')
    
    ball_x = np.array([0.0, 0.0])
    ball_y = np.array([0.2, 0.4])
    ax.plot(ball_x, ball_y, color = 'C0')
    
    ball = Circle((0.0, 0.4), radius = 0.02, color = 'C0')
    ax.add_artist(ball)
    
    paddle_x = np.array([-0.15 / 2.0 * np.sin(deg2rad(ball_traj_angle)),
                         0.15 / 2.0 * np.sin(deg2rad(ball_traj_angle))])
    paddle_y = np.array([0.75 - 0.15 / 2.0 * np.cos(deg2rad(ball_traj_angle)),
                         0.75 + 0.15 / 2.0 * np.cos(deg2rad(ball_traj_angle))])
    ax.plot(paddle_x, paddle_y, color = 'C1', linewidth = 4.0)
    
    paddle_line_x = np.array([0.0, 0.0])
    paddle_line_y = np.array([0.65, 0.85])
    ax.plot(paddle_line_x, paddle_line_y, color = 'C1', linewidth = 1.0)
    
    theta = np.linspace(0, ball_traj_angle, 101)
    theta_x = -0.15 / 4.0 * np.sin(deg2rad(theta))
    theta_y = 0.75 - 0.15 / 4.0 * np.cos(deg2rad(theta))
    ax.plot(theta_x, theta_y, color = 'C1', linewidth = 1.0)
    
    ax.arrow(0.0, 0.4, 0.0, 0.1, color = 'C0', width = 0.003, head_width = 0.015)
    
    ax.text(0.07, 0.65, 
            r'$\theta = {:.1f}$'.format(ball_traj_angle),
            fontsize = 24,
            color = 'C1')
    
    ax.text(0.07, 0.5,
            r'$v = {:.2f} m/s$'.format(ball_traj_speed),
            fontsize = 24,
            color = 'C0')
    
    ax.set_xlim([-0.1, 0.3])
    ax.set_xticks([])
    ax.set_yticks([])
    
    for spine in ax.spines.values():
        spine.set_visible(False)
    
    plt.show()
    
    print("Now, tell me about the outcome of the experiment:")

    v_ball_after = widgets.BoundedFloatText(
    value = 10.0,
    min = 0.0,
    description = "Velocity of ball (m/s):",
    continuous_update = False,
    style = style)

    angle_ball_after = widgets.BoundedFloatText(
    value = 30.0,
    min = -180.0 + ball_traj_angle,
    max = ball_traj_angle,
    description = "Angle of ball's path (deg):",
    continuous_update = False,
    style = style)

    ui2 = widgets.VBox([v_ball_after, angle_ball_after])

    def shot_outcome(v_ball_after, angle_ball_after):
        fig, ax = plt.subplots(figsize = (6, 6))
        
        ax.axis('equal')
        
        ball_before_x = np.array([0.0, 0.0])
        ball_before_y = np.array([0.3, 0.75])
        ax.plot(ball_before_x, ball_before_y, color = 'C0', linewidth = 1.5)
        
        ax.plot(paddle_x, paddle_y, color = 'C1', linewidth = 4.0)
        
        ball_after_x = np.array([0.0, -0.3 * np.sin(deg2rad(angle_ball_after))])
        ball_after_y = np.array([0.75, 0.75 - 0.3 * np.cos(deg2rad(angle_ball_after))])
        ax.plot(ball_after_x, ball_after_y, color = 'C0', linewidth = 1.5)
        
        beta = np.linspace(0, angle_ball_after, 101)
        beta_x = -0.3 / 2.0 * np.sin(deg2rad(beta))
        beta_y = 0.75 - 0.3 / 2.0 * np.cos(deg2rad(beta))
        ax.plot(beta_x, beta_y, color = 'C0', linewidth = 0.8)
        
        ball = Circle((ball_after_x[-1], ball_after_y[-1]), radius = 0.02, color = 'C0')
        ax.add_artist(ball)
        
        ax.arrow(ball_after_x[-1], ball_after_y[-1], 
                 -0.1 * np.sin(deg2rad(angle_ball_after)), 
                 -0.1 * np.cos(deg2rad(angle_ball_after)), 
                 color = 'C0', width = 0.003, head_width = 0.015)
        
        ax.set_xlim([-0.5, 0.5])
        ax.set_ylim([0.2, 1.2])
        
        ax.set_xticks([])
        ax.set_yticks([])
        
        for spine in ax.spines.values():
            spine.set_visible(False)
        
        plt.show()
        
        
        # Now we do MATH to bring it back to the world reference frame
        
#         print("v_ball_after")
#         print(v_ball_after)
#         print("angle_ball_after")
#         print(angle_ball_after)
        v_ball_after_in_p = np.matrix([[-v_ball_after * np.cos(deg2rad(angle_ball_after))],
                                       [ v_ball_after * np.sin(deg2rad(angle_ball_after))]])
        
#         print("v_ball_after_in_p")
#         print(v_ball_after_in_p)
        closed_paddle_rot_mat = np.matrix([[np.cos(deg2rad(closed_paddle)), -np.sin(deg2rad(closed_paddle))],
                                           [np.sin(deg2rad(closed_paddle)),  np.cos(deg2rad(closed_paddle))]])
#         print("closed_paddle_rot_mat")
#         print(closed_paddle_rot_mat)
        
        v_ball_after_in_rot_p = closed_paddle_rot_mat * v_ball_after_in_p
#         print("v_ball_after_in_rot_p")
#         print(v_ball_after_in_rot_p)
        
        v_ball_after_in_w = v_ball_after_in_rot_p + paddle_vec
#         print("paddle_vec")
#         print(paddle_vec)
#         print("v_ball_after_in_w")
#         print(v_ball_after_in_w)
        
        speed_ball_after_in_w = (v_ball_after_in_w.T * v_ball_after_in_w)[0,0]**0.5
        
        angle_ball_after_in_w = 180.0 / np.pi * np.arctan(- v_ball_after_in_w[1,0] / v_ball_after_in_w[0,0])
        
#         print(v_ball_after_in_w)
#         print(speed_ball_after_in_w)
#         print(angle_ball_after_in_w)
#         print("something is wrong with this angle")
        
        
        print("Based on that, here's what you would expect from the shot:")
        
        fig, ax = plt.subplots(figsize = (8, 4))
    
        ax.axis('equal')
    
        #ax.plot(x_table, y_table, color = 'black', linewidth = 2.0)
    
        #ax.plot(x_ball_prev, y_ball_prev, color = 'C0')

        ax.plot(x_ball_after, y_ball_after, color = 'C0')
        #print(x_paddle)
        #print(y_paddle)
        ax.plot(x_paddle, y_paddle, color = 'C1')

        ax.plot(x_bat, y_bat, color = 'C1', linewidth = 4.0)

        t_f = np.linspace(0.0, 0.25, 101)
        x_f = x_p + v_ball_after_in_w[0,0] * t_f
        y_f = y_p + v_ball_after_in_w[1,0] * t_f - 9.8 / 2.0 * t_f**2.0
        ax.plot(x_f, y_f, color = 'C0')
        
        ball = Circle((x_f[-1], y_f[-1]), radius = 0.02, color = 'C0', zorder = 2)
        ax.add_artist(ball)
        
        ax.set_xticks([])
        ax.set_yticks([])
        
        for spine in ax.spines.values():
            spine.set_visible(False)
        
        plt.show()
        
        print('Final ball velocity in world: ' + '{:.2f}'.format(speed_ball_after_in_w) + ' m/s')
        print('Final ball angle above the horizontal: ' + '{:.1f}'.format(angle_ball_after_in_w) + ' deg')
        
    
    out2 = widgets.interactive_output(shot_outcome, {'v_ball_after': v_ball_after,
                                                 'angle_ball_after': angle_ball_after})
    display(ui2, out2)
    
    
    
    
out = widgets.interactive_output(shot_setup, {'v_ball': v_ball, 
                                              'angle_ball': angle_ball,
                                              'v_paddle': v_paddle,
                                              'angle_paddle': angle_paddle,
                                              'closed_paddle': closed_paddle})
display(ui, out)

Output()

In [5]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')